In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import numpy as np
import scipy.stats as stats
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib as mpl

from bgspy.genome import Genome
from bgspy.data import GenomeData

In [3]:
g = Genome('hg38', seqlens_file="../data/annotation/hg38_seqlens.tsv", chroms=['chr1'])
g.load_annot("../data/annotation/conserved_phastcons_thresh10.bed.gz")
g.load_recmap("../data/annotation/HapMapII_GRCh37_liftedOverTo_Hg38/genetic_map_Hg38_chr10.txt")

load_bed_annotation(): ignored chr10
ignoring HapMap header...
parsing recmap as HapMap formatted (chrom, end, rate)
RecMap._readmap() ignored chr10


In [4]:
gd = GenomeData(g)
gd.load_counts_dir('../data/hg1k_counts/')
gd.load_neutral_masks('../data/annotation/neutral_cds_phastcons.bed')
gd.load_accessibile_masks('../data/annotation/no_centro.bed')